In [ ]:
# !pip install optuna

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

import optuna

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    MODEL_PATH = 'microsoft/deberta-xlarge'
    TOKENIZER_PATH = 'microsoft/deberta-xlarge'
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'deberta-xlarge'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [10]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [11]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [12]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [13]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_bos_token',
 '_build_conversation_input_ids',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_push_to_hub',
 '_save_pretrained',
 '_sep_token',
 '_tokenizer',
 '_unk_token',
 'add_pr

In [14]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [15]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [16]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [17]:
from transformers import AutoModelForSequenceClassification

class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.MODEL_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModelForSequenceClassification.from_pretrained(cfg.MODEL_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['hidden_states']
        last_layer_hidden_states = hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [18]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['pooler.dense.weight', 'po

In [19]:
import re

for i, (name, param) in enumerate(sample_model.named_parameters()):
    if(name.find('layer') > -1):
        layer_name = re.sub(r'.+(layer\.\d+).+', r'\1', name)

In [20]:
for i, (name, param) in enumerate(sample_model.named_parameters()):
    print(i, name, param.size())

0 transformer_model.deberta.embeddings.word_embeddings.weight torch.Size([50265, 1024])
1 transformer_model.deberta.embeddings.LayerNorm.weight torch.Size([1024])
2 transformer_model.deberta.embeddings.LayerNorm.bias torch.Size([1024])
3 transformer_model.deberta.encoder.layer.0.attention.self.q_bias torch.Size([1024])
4 transformer_model.deberta.encoder.layer.0.attention.self.v_bias torch.Size([1024])
5 transformer_model.deberta.encoder.layer.0.attention.self.in_proj.weight torch.Size([3072, 1024])
6 transformer_model.deberta.encoder.layer.0.attention.self.pos_proj.weight torch.Size([1024, 1024])
7 transformer_model.deberta.encoder.layer.0.attention.self.pos_q_proj.weight torch.Size([1024, 1024])
8 transformer_model.deberta.encoder.layer.0.attention.self.pos_q_proj.bias torch.Size([1024])
9 transformer_model.deberta.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024])
10 transformer_model.deberta.encoder.layer.0.attention.output.dense.bias torch.Size([1024])
11 trans

In [21]:
sample_input_ids = torch.randint(0, 1000, [8, 248])
sample_attention_mask = torch.randint(0, 1000, [8, 248])

In [22]:
sample_model(sample_input_ids, sample_attention_mask)[1].shape

torch.Size([8, 1024])

In [23]:
torch.sum(torch.randn([8, 496, 768]), axis=1)

tensor([[-21.8222,  11.9218,  -3.9006,  ...,  -7.6640,  18.5257,  11.2347],
        [  9.2061, -20.5722,  -2.4569,  ...,  -2.0841,  16.7695,  31.1698],
        [ 18.7644,  11.5316,   1.3392,  ...,  -7.7326,   9.6947, -12.7335],
        ...,
        [  9.7215,   8.6309,  10.5881,  ...,  -0.4056, -30.2846,  15.3793],
        [ -6.3272,  17.1342,   5.5680,  ..., -29.6354,  -5.1623,   6.1300],
        [ -3.0353, -24.1111,  28.8121,  ..., -22.2144,  -1.6490,   0.2607]])

### Evaluation and Prediction

In [24]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [25]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [26]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [27]:
5e-5 / 2.5, 5e-5 / 0.5, 5e-5

(2e-05, 0.0001, 5e-05)

In [28]:
def create_optimizer(model, base_lr=5e-5, last_lr=None):
    named_parameters = list(model.named_parameters())
    attention_param_start = 776
    regressor_param_start = 780
    roberta_parameters = named_parameters[:attention_param_start]
    attention_parameters = named_parameters[attention_param_start:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    if last_lr is not None:
        parameters.append({"params": attention_group, "lr": last_lr})
        parameters.append({"params": regressor_group, "lr": last_lr})
    else:
        parameters.append({"params": attention_group})
        parameters.append({"params": regressor_group})
        
    # Change on different models
    layer_low_threshold = 275
    layer_middle_threshold = 571
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5 # 2e-05
        if layer_num >= layer_middle_threshold:
            lr = base_lr / 0.5 # 1e-4
        elif layer_num >= layer_low_threshold:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [29]:
sample_optimizer = create_optimizer(sample_model)

In [30]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [31]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [32]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [33]:
class Trainer():
    def __init__(self, scaler, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.scaler, self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            scaler, model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        val_rmse_list = []
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                # Casts operations to mixed precision
                with torch.cuda.amp.autocast():
                    pred, _ = self.model(input_ids, attention_mask)
                    mse = mse_loss(pred.flatten(), target)
                    
                self.scaler.scale(mse).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
                
#                 mse.backward()
#                 self.optimizer.step()
                
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, self.model, self.model_path)
                    val_rmse_list.append(val_rmse)
                    start = time.time()
                # Finish early on condition
                if epoch > 0 and best_val_rmse > 0.6 or (len(val_rmse_list) > 5 and np.array(val_rmse_list).mean() > 1.0):
                    return best_val_rmse
                
                step += 1
        return best_val_rmse

In [34]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = list(kfold.split(train_df))

### Main Training

In [35]:
def train_fold(base_lr, last_lr, fold=0, epochs=3):
    
    print(f'##### Using fold {fold}')
    print(f'##### Using base_lr {base_lr} last_lr {last_lr}')
    
    model_path = cfg.MODEL_FOLDER/f"{cfg.model_name.replace('/', '_')}_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_indices, val_indices = splits[fold]
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
    
    optimizer = create_optimizer(model, base_lr=base_lr, last_lr=last_lr)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    scaler = torch.cuda.amp.GradScaler()
    
    trainer = Trainer(scaler, model, model_path, train_loader, val_loader, optimizer, scheduler=scheduler, num_epochs=epochs)
    rmse_val = trainer.train()
    tokenizer.save_pretrained(str(model_path.parent))
    
    return rmse_val

In [36]:
# Best results
# Fold 0: { 'base_lr': 4.3596909535440914e-05, 'last_lr': 0.0004188473213340135, 'epochs': 4 } Best value: 0.4722290635108948
# Fold 1: {'base_lr': 3.093409522252196e-05, 'last_lr': 0.0004074485086437216, 'epochs': 4} Best is trial 9 with value: 0.4512692391872406
# Fold 2: {'base_lr': 5.9004819673113075e-05, 'last_lr': 0.0003701804156340247, 'epochs': 5}   Best value:  0.46230143308639526
# Fold 3: {'base_lr': 3.091841397163233e-05, 'last_lr': 0.00010409734625896974, 'epochs': 4}. Best value:  0.474480539560318
# Fold 4: {'base_lr': 3.2314567372708084e-05, 'last_lr': 8.327155005618419e-05, 'epochs': 4}. Best is trial 0 with value: 0.45970267057418823
# Fold 5: {'base_lr': 3.5379120180791935e-05, 'last_lr': 0.00021137535166837663, 'epochs': 5}. Best is trial 0 with value: 0.46050626039505005

In [37]:
lr_list = [
    { 'base_lr': 4.3596909535440914e-05, 'last_lr': 0.0004188473213340135, 'epochs': 4 },
    {'base_lr': 3.093409522252196e-05, 'last_lr': 0.0004074485086437216, 'epochs': 4},
    {'base_lr': 5.9004819673113075e-05, 'last_lr': 0.0003701804156340247, 'epochs': 5},
    {'base_lr': 3.091841397163233e-05, 'last_lr': 0.00010409734625896974, 'epochs': 4},
    {'base_lr': 3.2314567372708084e-05, 'last_lr': 8.327155005618419e-05, 'epochs': 4},
    {'base_lr': 3.5379120180791935e-05, 'last_lr': 0.00021137535166837663, 'epochs': 5}  
]

In [38]:
%%time

rmse_values = []
for i in range(len(list(splits))):
    fold = i
    lrs = lr_list[fold]
    rmse_val = train_fold(lrs['base_lr'], lrs['last_lr'], fold=fold, epochs=lrs['epochs'])
    print(f'Final RMSE: {rmse_val}')
    rmse_values.append(rmse_val)

##### Using fold 0
##### Using base_lr 4.3596909535440914e-05 last_lr 0.0004188473213340135


Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['classifier.bias', 'classi


16 steps took 18.1 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8007 New best_val_rmse: 0.8007

16 steps took 13.2 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7216 New best_val_rmse: 0.7216

16 steps took 13.3 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.8994 Still best_val_rmse: 0.7216 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.8762 Still best_val_rmse: 0.7216 (from epoch 0)

16 steps took 13.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6523 New best_val_rmse: 0.6523

16 steps took 13.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.782 Still best_val_rmse: 0.6523 (from epoch 0)

16 steps took 13.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6558 Still best_val_rmse: 0.6523 (from epoch 0)

16 steps took 13.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5925 New best_val_rmse: 0.5925

16 steps took 13.4 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6099 Still best_val_rmse: 0.5925 (from epoch 0)

16 steps took 14.2 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5123

Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['classifier.bias', 'classi


16 steps took 16.2 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8201 New best_val_rmse: 0.8201

16 steps took 13.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7671 New best_val_rmse: 0.7671

16 steps took 13.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6906 New best_val_rmse: 0.6906

16 steps took 13.3 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6273 New best_val_rmse: 0.6273

16 steps took 13.3 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5923 New best_val_rmse: 0.5923

16 steps took 13.3 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5623 New best_val_rmse: 0.5623

16 steps took 13.3 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.56 New best_val_rmse: 0.56

16 steps took 13.5 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6405 Still best_val_rmse: 0.56 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.562 Still best_val_rmse: 0.56 (from epoch 0)

16 steps took 14.1 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5441 New best_val_rmse: 0.5441

16 steps took 13.4 seconds
Epoc

...


16 steps took 15.7 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8222 New best_val_rmse: 0.8222

16 steps took 13.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6415 New best_val_rmse: 0.6415

16 steps took 13.3 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7187 Still best_val_rmse: 0.6415 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6958 Still best_val_rmse: 0.6415 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6556 Still best_val_rmse: 0.6415 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.58 New best_val_rmse: 0.58

16 steps took 13.3 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.7503 Still best_val_rmse: 0.58 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6572 Still best_val_rmse: 0.58 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5782 New best_val_rmse: 0.5782

16 steps took 14.2 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5185 New be

Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['classifier.bias', 'classi


16 steps took 15.6 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9454 New best_val_rmse: 0.9454

16 steps took 13.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7717 New best_val_rmse: 0.7717

16 steps took 13.3 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7838 Still best_val_rmse: 0.7717 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6301 New best_val_rmse: 0.6301

16 steps took 13.3 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5986 New best_val_rmse: 0.5986

16 steps took 13.3 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5716 New best_val_rmse: 0.5716

16 steps took 13.3 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5557 New best_val_rmse: 0.5557

16 steps took 13.3 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5844 Still best_val_rmse: 0.5557 (from epoch 0)

16 steps took 13.6 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6491 Still best_val_rmse: 0.5557 (from epoch 0)

16 steps took 14.2 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.4925 New best_val_rmse: 0.4925

8 ste

Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['classifier.bias', 'classi


16 steps took 15.7 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.7172 New best_val_rmse: 0.7172

16 steps took 13.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7892 Still best_val_rmse: 0.7172 (from epoch 0)

16 steps took 13.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6249 New best_val_rmse: 0.6249

16 steps took 13.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7399 Still best_val_rmse: 0.6249 (from epoch 0)

16 steps took 13.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6589 Still best_val_rmse: 0.6249 (from epoch 0)

16 steps took 13.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5633 New best_val_rmse: 0.5633

16 steps took 13.3 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5695 Still best_val_rmse: 0.5633 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5865 Still best_val_rmse: 0.5633 (from epoch 0)

16 steps took 13.3 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5579 New best_val_rmse: 0.5579

16 steps took 14.1 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.545

In [39]:
f'mean RMSE values: {np.mean(np.array(rmse_values))}'

'mean RMSE values: 0.4634148180484772'

### Verify the model

In [38]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [39]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [40]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [41]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1


Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['pooler.dense.weight', 'po

Model 2


Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['pooler.dense.weight', 'po

Model 3


...

Model 5


Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['pooler.dense.weight', 'po

Model 6


Some weights of the model checkpoint at microsoft/deberta-xlarge were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge and are newly initialized: ['pooler.dense.weight', 'po

CPU times: user 50.5 s, sys: 19.4 s, total: 1min 9s
Wall time: 1min 39s


In [42]:
from transformers import DebertaTokenizer

tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = DebertaTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}")
    tokenizers.append(tokenizer)

In [43]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [44]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [45]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [46]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [47]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [48]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [49]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.31464171601263147
Fold 1 (2361,) (472,)
rmse_score 0.27859509058687465
Fold 2 (2361,) (472,)
rmse_score 0.2838705330645049
Fold 3 (2361,) (472,)
rmse_score 0.2632614086820568
Fold 4 (2361,) (472,)
rmse_score 0.2904974716258125
Fold 5 (2361,) (472,)
rmse_score 0.2922923251303494
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.2964359469540683
Fold 1 (2361,) (472,)
rmse_score 0.28675202793265897
Fold 2 (2361,) (472,)
rmse_score 0.2706928974295034
Fold 3 (2361,) (472,)
rmse_score 0.28467305998990955
Fold 4 (2361,) (472,)
rmse_score 0.27693783263754496
Fold 5 (2361,) (472,)
rmse_score 0.2998636986220618
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.273803042827445
Fold 1 (2361,) (472,)
rmse_score 0.24919983849443164
Fold 2 (2361,) (472,)
rmse_score 0.24640098125587462
Fold 3 (2361,) (472,)
rmse_score 0.2642489696681386
Fold 4 (2361,) (472,)
rmse_score 0.2653003416579225
Fold 5 (2361,) (472,)
rmse_score 0.28370421129281065
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.2844604937236611
Fold 1 (2361,) (472,)
rmse_score 0.2771335340794905
Fold 2 (2361,) (472,)
rmse_score 0.28104751964700075
Fold 3 (2361,) (472,)
rmse_score 0.24509229264781246
Fold 4 (2361,) (472,)
rmse_score 0.2852742256236505
Fold 5 (2361,) (472,)
rmse_score 0.28913895665966094
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.28175397357882287
Fold 1 (2361,) (472,)
rmse_score 0.2659750669296263
Fold 2 (2361,) (472,)
rmse_score 0.25115957353869156
Fold 3 (2361,) (472,)
rmse_score 0.25728498559756635
Fold 4 (2361,) (472,)
rmse_score 0.28459032169210746
Fold 5 (2361,) (472,)
rmse_score 0.28134385940943885
FINAL RMSE score 0.2768475398997376
CPU times: user 4min 39s, sys: 1min 17s, total: 5min 56s
Wall time: 5min 54s


In [50]:
final_kernel_predictions_means

[[0.001354966581732184],
 [0.002679879202131139],
 [0.0014281085903620552],
 [-0.0005655793859269693],
 [0.0022437018163661013]]

In [51]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [52]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.19813154, 0.19836642, 0.20236074, 0.19996804, 0.20117325])

In [53]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [54]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.8827289480715281)

In [55]:
final_scores_flat

array([-0.37493544, -0.58117305, -0.42583862, -2.06807065, -1.8151781 ,
       -1.21629038,  0.30367699])

In [56]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(-0.07711321535133042, -0.015422643070266084)

In [57]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.452049
1,f0953f0a5,-0.658286
2,0df072751,-0.502952
3,04caf4e0c,-2.145184
4,0e63f8bea,-1.892291
5,12537fe78,-1.293404
6,965e592c0,0.226564


### Prepare Packaging

In [58]:
cfg.model_name

'deberta-xlarge'

In [59]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [60]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/deberta-xlarge/best')

In [61]:
cfg.NUM_FOLDS

6

In [62]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [63]:
bestmodels

[PosixPath('/home/commonlit/models/deberta-xlarge_1'),
 PosixPath('/home/commonlit/models/deberta-xlarge_2'),
 PosixPath('/home/commonlit/models/deberta-xlarge_3'),
 PosixPath('/home/commonlit/models/deberta-xlarge_4'),
 PosixPath('/home/commonlit/models/deberta-xlarge_5'),
 PosixPath('/home/commonlit/models/deberta-xlarge_6')]

In [64]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    i = i + 1
    best_model_file = f'{best_model}/model_{i}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer_config.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'tokenizer.json')

        special_tokens = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/special_tokens_map.json'))
        assert special_tokens.exists()
        copyfile(special_tokens, tokenizer_path/'special_tokens_map')
        
        merges = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/merges.txt'))
        assert merges.exists()
        copyfile(merges, tokenizer_path/'merges.txt')
        
        vocab = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/vocab.json'))
        assert vocab.exists()
        copyfile(vocab, tokenizer_path/'vocab.json')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [65]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/deberta-xlarge/best_models.zip'

In [66]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip  dataset-metadata.json  lm.zip


In [67]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/deberta-xlarge.yaml': No such file or directory


In [68]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [69]:
!du -h {MODELS_PATH/cfg.model_name}/*

2.5M	/home/commonlit/models/deberta-xlarge/best/tokenizer-1
2.5M	/home/commonlit/models/deberta-xlarge/best/tokenizer-2
2.5M	/home/commonlit/models/deberta-xlarge/best/tokenizer-3
2.5M	/home/commonlit/models/deberta-xlarge/best/tokenizer-4
2.5M	/home/commonlit/models/deberta-xlarge/best/tokenizer-5
2.5M	/home/commonlit/models/deberta-xlarge/best/tokenizer-6
17G	/home/commonlit/models/deberta-xlarge/best
16G	/home/commonlit/models/deberta-xlarge/best_models.zip
4.0K	/home/commonlit/models/deberta-xlarge/dataset-metadata.json
2.9G	/home/commonlit/models/deberta-xlarge/lm
2.7G	/home/commonlit/models/deberta-xlarge/lm.zip


In [70]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/deberta-xlarge/lm.zip'

In [77]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/deberta-xlarge/dataset-metadata.json


In [78]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [79]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [80]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-deberta-xlarge",
  "id": "gilfernandes/commonlit-deberta-xlarge",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [81]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [72]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 15.7G/15.7G [26:13<00:00, 10.7MB/s]
Upload successful: best_models.zip (16GB)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 2.61G/2.61G [04:23<00:00, 10.6MB/s]
Upload successful: lm.zip (3GB)
Skipping folder: best; use '--dir-mode' to upload folders
Skipping folder: lm; use '--dir-mode' to upload folders
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gilfernandes/commonlit-deberta-xlarge


In [71]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with vocab.json" -d

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 15.7G/15.7G [25:38<00:00, 10.9MB/s]
Upload successful: best_models.zip (16GB)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 2.61G/2.61G [04:25<00:00, 10.5MB/s]
Upload successful: lm.zip (3GB)
Skipping folder: best; use '--dir-mode' to upload folders
Skipping folder: lm; use '--dir-mode' to upload folders
Dataset version is being created. Please check progress at /api/v1/datasets/status/gilfernandes/commonlit-deberta-xlarge


In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)